# Text classification

This week we are moving from  classifiyng characteristics of single words to classifying whole texts. However, instead of trying to classify the sentiment of a text, we will be classifying whether texts are toxic or not. We are using the toxi-text dataset from huggingface. You can find more information about the dataset [here](https://huggingface.co/datasets/FredZhang7/toxi-text-3M). Try to get an overview of:
- what kind of data it contains
- where the data comes from
- what the labels mean

If you prefer not to read toxic text you can use [this](https://huggingface.co/datasets/stanfordnlp/imdb) dataset instead which contains imdb reviews and sentiment classification labels - or any other dataset you prefer :-)

## Install packages

In [ ]:
!pip install nltk
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install gensim
!pip install scikit-learn
!pip install fsspec
!pip install huggingface-hub

## Import packages

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.linear_model import LogisticRegression
import gensim.downloader
import numpy as np

## Load data

The dataset is very large and multilingual, so for efficiency's sake we will only use a smaller, English subset of the data. We don't have to split the data into training and test sets because the dataset already has a test set which is saved in a separate file.

In [3]:
df = pd.read_csv("hf://datasets/FredZhang7/toxi-text-3M/train/multilingual-train-deduplicated.csv", nrows=100000)

In [ ]:
df = df[df.lang == 'en']
df

## Preprocessing

The sklearn bag-of-words model expects the data to be a sequence of strings:

In [ ]:
texts = df["text"].tolist()
texts

## Bag-of-words 

One of the simplest way to represent a document is a bag-of-words model. This model represents a document as a set of words, ignoring the order of the words. The model is implemented in the `CountVectorizer` class in sklearn.

In [10]:
vectorizer = CountVectorizer(stop_words = 'english')
features = vectorizer.fit_transform(texts)

In [7]:
features.shape

(86996, 155736)

The shape of the matrix should correspond to the number of documents and the number of unique words in the dataset. The value of each cell should correspond to the number of times the word appears in the document.

In [11]:
vectorizer.vocabulary_

{'saved': 121140,
 'lives': 83649,
 'spent': 129208,
 'children': 32130,
 'liberal': 82698,
 'madness': 85834,
 'playing': 106903,
 'theatre': 136769,
 'near': 95583,
 'agree': 13523,
 'say': 121217,
 'workers': 151166,
 'expensive': 52657,
 'live': 83618,
 'vancouver': 145307,
 'maybe': 88189,
 'happier': 64751,
 'moving': 93083,
 'slightly': 127008,
 'downtown': 46589,
 'extreme': 52986,
 'toronto': 138984,
 'observation': 99041,
 'exists': 52531,
 'unequal': 142973,
 'share': 124460,
 'state': 130569,
 'monies': 92243,
 'residents': 116687,
 'urban': 144351,
 'defensive': 41709,
 'hear': 65667,
 'presently': 109284,
 'corporations': 37261,
 'pay': 103886,
 'income': 71270,
 'taxes': 135382,
 'individual': 71609,
 'pays': 103933,
 'noticed': 97908,
 'funded': 58337,
 'bicycle': 23511,
 'paths': 103668,
 'road': 118256,
 'maintenance': 86190,
 'defunct': 41836,
 'docks': 45777,
 'powerful': 108582,
 'politicians': 107587,
 'pet': 105255,
 'projects': 110188,
 'office': 99440,
 'buildi

In [12]:
len(vectorizer.vocabulary_)

155423

In [10]:
len(texts)

86996

Lastly, we need to create a list of the labels:

In [13]:
y = df.is_toxic.tolist()

In [ ]:
y

## Training a model

Now we can train a model to classify the toxicity of the texts. I will use a simple logistic regression model, but feel free to swap it out for any other model you prefer.

In [14]:
clf = LogisticRegression(random_state=42, 
                         max_iter = 1000, 
                         verbose = True)

In [15]:
clf.fit(features, y)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       155424     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  3.81995D-01


 This problem is unconstrained.



At iterate   50    f=  1.22826D-01    |proj g|=  1.88685D-03

At iterate  100    f=  1.09791D-01    |proj g|=  4.51174D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
*****    135    143      1     0     0   9.479D-05   1.084D-01
  F =  0.10842171319289146     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


LogisticRegression(max_iter=1000, random_state=42, verbose=True)

In [16]:
clf.score(features, y)

0.9741137523564302


removing lowercasing doesnt affect the score 
stop words = english - improves the score : 0.9741137523564302


Now try to take a look at the documentation for the [Countvectorizer](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Try to change the parameters of the model and see how it affects the performance of the model:
- try to remove lowercasing and see how it affects performance
- try to add stopwords to the model
- try to see if you can find a parameter that can be used as an alternative to stopword removal
- try to change the ngram_range parameter
- try to change how the model tokenises the text by changing the token_pattern parameter (hint: use a regex generator)

## tf-idf

Another simple, yet slightly more advanced model is the tf-idf model. This model is also implemented in sklearn. The model is implemented in the `TfidfVectorizer` class in sklearn.

- try to create tfidf features from our texts and run the classifier again
- take a look at the [documentation](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) and try to change the parameters of the model and see how it affects the performance of the model

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words = "english")
features = tfidf_vectorizer.fit_transform(texts)
y = df.is_toxic.tolist()
clf.fit(features, y)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       155424     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  3.81995D-01


 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
*****     42     46      1     0     0   3.389D-05   2.092D-01
  F =  0.20915667146650760     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


LogisticRegression(max_iter=1000, random_state=42, verbose=True)

In [25]:
clf.score(features, y) 
# with stop_words = english 0.9331118672122857
# without stop_words 0.9326980550829923

0.9331118672122857

## Document embeddings

A much more nuanced way to represent text is through embeddings. However, most machine learning models require a fixed-size input, so we need to find a way to represent the whole document as a fixed-size vector. One way to do this is to use the average of the word embeddings of the words in the document. We will use the pre-trained word embeddings from the GloVe model. However, using word embeddings requires us to split the documents into individual words. We will use the nltk library to do this, but there are both simpler and more advanced ways to do this. The simplest method would be to split the documents by spaces, while a more advanced method would be to use a tokenizer that is aware of the structure of the language, like the one in the [spacy](https://spacy.io/api/tokenizer) library.

In [26]:
import nltk

nltk.download('punkt')

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/ucloud/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


If we try to tokenise the first of the texts, we get:

In [27]:
word_tokenize(texts[0], 
              language='english', 
              preserve_line=True)

['Saved',
 'lives',
 ',',
 'and',
 'spent',
 'for',
 'all',
 'of',
 'their',
 'children',
 "'s",
 'lives.',
 'LIberal',
 'Madness',
 ',',
 'playing',
 'at',
 'a',
 'theatre',
 'near',
 'you',
 '.']

Now we can load the embeddings and match our tokenised words to the embeddings:

In [28]:
embeddings = gensim.downloader.load("glove-wiki-gigaword-300")

[==================================================] 100.0% 376.1/376.1MB downloaded


In [29]:
def get_embeddings(text):
    return [embeddings[word] for word in word_tokenize(text, language='english', preserve_line=True) if word in embeddings.key_to_index]

In [31]:
text_embeddings = [get_embeddings(text) for text in texts]

In [ ]:
print(len(text_embeddings[0]))
print(len(text_embeddings[0][0]))

In [ ]:
print(len(text_embeddings[1]))
print(len(text_embeddings[1][0]))

We see that though the individual word embeddings have to same number of dimensions, the document embeddings have different sizes. We can fix this by taking the average of the word embeddings:

In [85]:
mean_embeddings = [np.mean(embedding, axis=0) for embedding in text_embeddings]

In [ ]:
mean_embeddings[0].shape

Now you have mean document embeddings that you can use to classify the texts!

- try to classify the texts using the average of the word embeddings of the words in the text
- try lowercasing the words before creating the embeddings
- try removing stopwords or punctuation beore creating the embeddings
- try using another classifier
- try to use all the languages in the dataset and see how it affects the performance of the model

In [ ]:
clf = LogisticRegression(random_state=42, 
                         max_iter = 1000, 
                         verbose = True)
clf.fit(mean_embeddings, y)